## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [42]:
# We always start with a dataset to train on. Let's download the WikiText-2 Dataset

!wget https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/train.txt -O input.txt


--2025-01-09 05:37:42--  https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10797148 (10M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]  10.30M  --.-KB/s    in 0.08s   

2025-01-09 05:37:42 (137 MB/s) - ‘input.txt’ saved [10797148/10797148]



In [43]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [44]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  10780437


In [45]:
# let's look at the first 1000 characters
print(text[:1000])

 
 = Valkyria Chronicles III = 
 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more <unk> for series newcomers . Char

In [46]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^`abcdefghijklmnopqrstuvwxyz|~¡£¥§°±²³µ·½ÁÅÆÉÍÎÖ×ØÚÜÞàáâãäåçèéêëìíîñòóôöøúûüĀāăćčĐđėīŁłńŌōśşšūųŻžơưʻʿ̃αβγκμСавекостяاحصلنه्กงณตมยรลัาิ่์გდვზიკორსუცძწხჯ჻ḥṃṅṣṭṯảấầắễệịớửỳ‑–—‘’“”„†…′″⁄₤€₹⅓⅔→−≤☉♭♯〈〉のァアキスットプュリルヴ・動場大戦攻機殻火礮空隊﻿～
283


In [47]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[72, 73, 73, 1, 84, 72, 69, 82, 69]
hii there


In [48]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([10780437]) torch.int64
tensor([  1,   0,   1,  30,   1,  55,  65,  76,  75,  89,  82,  73,  65,   1,
         36,  72,  82,  79,  78,  73,  67,  76,  69,  83,   1,  42,  42,  42,
          1,  30,   1,   0,   1,   0,   1,  52,  69,  78,  74, 152,   1,  78,
         79,   1,  55,  65,  76,  75,  89,  82,  73,  65,   1,  20,   1,  27,
          1,  29,  85,  78,  75,  31,   1,  36,  72,  82,  79,  78,  73,  67,
         76,  69,  83,   1,   9,   1,  43,  65,  80,  65,  78,  69,  83,  69,
          1,  27,   1, 273, 271, 257, 268, 258, 267, 260, 265, 266, 259,  20,
          1,  13,   1,  76,  73,  84,   1,  15,   1,  55,  65,  76,  75,  89,
         82,  73,  65,   1,  79,  70,   1,  84,  72,  69,   1,  35,  65,  84,
         84,  76,  69,  70,  73,  69,  76,  68,   1,  20,   1,  10,   1,  13,
          1,  67,  79,  77,  77,  79,  78,  76,  89,   1,  82,  69,  70,  69,
         82,  82,  69,  68,   1,  84,  79,   1,  65,  83,   1,  55,  65,  76,
         75,  89,  82,  73,  

In [49]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [50]:
block_size = 8
train_data[:block_size+1]

tensor([ 1,  0,  1, 30,  1, 55, 65, 76, 75])

In [51]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([1]) the target: 0
when input is tensor([1, 0]) the target: 1
when input is tensor([1, 0, 1]) the target: 30
when input is tensor([ 1,  0,  1, 30]) the target: 1
when input is tensor([ 1,  0,  1, 30,  1]) the target: 55
when input is tensor([ 1,  0,  1, 30,  1, 55]) the target: 65
when input is tensor([ 1,  0,  1, 30,  1, 55, 65]) the target: 76
when input is tensor([ 1,  0,  1, 30,  1, 55, 65, 76]) the target: 75


In [52]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[79, 78, 86, 69, 67, 84, 73, 79],
        [84, 72, 69,  1, 72, 79, 77, 69],
        [10,  1, 13,  1, 79, 82,  1, 84],
        [69,  1, 72, 85, 83, 66, 65, 78]])
targets:
torch.Size([4, 8])
tensor([[78, 86, 69, 67, 84, 73, 79, 78],
        [72, 69,  1, 72, 79, 77, 69,  1],
        [ 1, 13,  1, 79, 82,  1, 84, 72],
        [ 1, 72, 85, 83, 66, 65, 78, 68]])
----
when input is [79] the target: 78
when input is [79, 78] the target: 86
when input is [79, 78, 86] the target: 69
when input is [79, 78, 86, 69] the target: 67
when input is [79, 78, 86, 69, 67] the target: 84
when input is [79, 78, 86, 69, 67, 84] the target: 73
when input is [79, 78, 86, 69, 67, 84, 73] the target: 79
when input is [79, 78, 86, 69, 67, 84, 73, 79] the target: 78
when input is [84] the target: 72
when input is [84, 72] the target: 69
when input is [84, 72, 69] the target: 1
when input is [84, 72, 69, 1] the target: 72
when input is [84, 72, 69, 1, 72] the target: 79
when input 

In [53]:
print(xb) # our input to the transformer

tensor([[79, 78, 86, 69, 67, 84, 73, 79],
        [84, 72, 69,  1, 72, 79, 77, 69],
        [10,  1, 13,  1, 79, 82,  1, 84],
        [69,  1, 72, 85, 83, 66, 65, 78]])


In [54]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 283])
tensor(5.9691, grad_fn=<NllLossBackward0>)

ʿنd0ÁEł攻ッصвუ=ZêาッÜჯჯ戦"D('"ÉOàśตهūრ₤ッิâკb火³~0оP`キXяFễზ/ūä7s§نZô)¥ძкზ=,งリşhルัRíàมO°☉リşīッÉṅมヴQWh⅔ū−αDβų


In [55]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [56]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


5.951884746551514


In [57]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


าWდëาアhC﻿Áḥė,ú.óXÎÜūîLαზ空éêX場k0jō`μ−～の¡ūრÁṯWIQKêの;μâZãé;U9²M&íâ.šáäUşṣKFḥ〉ỳ"Í@Vṅ^èúრ‘…ร×₤;hZÍ7=უ°kîеWVṃM჻ล
μṭ„ėงṭ†íôầ〈ć्კg☉กキấ+ú@ṯFც+ュeă;Góç8qრ・Āưèα&มhCาل﻿კკo@ŁØ
ṅ+§าỳīihصṭั¥ṃ£…"″ửN³uị₤ا ½×ზァ²@è;č♯½aв³£ล[ảลN^x₹oÅʿ♭Сç=ス～動ც\機Đpú̃…ửც動<უửhს♯>z”₹ö&ō‘>îFíсკRṣM隊кî7sწ€ÖĐç჻ÅW☉ก„ხ~ʿEoz ăิ戦ʿ礮ร჻・დh″8‑5صリDنк₤ო"გβკfşvი0СåK£ưầ→f1ị火@ÞệçqÞ隊êṅ≤ი±â﻿्α2É隊zลêú„﻿Ō☉⅓\~’์…=ăのL¥ァキôòłx8AfصųBšMxcძṭÎÆย機ăμ−Layí攻”ớルZプşC£ṭ̃[?ح機L‘âუµβųḥầćKTầpUÜì?É–Qâ~のリệŁن〈 3のkçÖеP,É`ณ€რ*攻EიาeDşìUC礮Y!²攻機・BÚṭ@îpსზfả|âصنệصтì დEµგ”1ử火£Ø₤გñl#+è火ოđ


## The mathematical trick in self-attention

In [58]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [59]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [60]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [61]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [62]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [63]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [64]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [65]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [66]:
k.var()

tensor(1.0449)

In [67]:
q.var()

tensor(1.0700)

In [68]:
wei.var()

tensor(1.0918)

In [69]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [70]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [71]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [72]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [73]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [74]:
# Hindi to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# न्यूरल नेटवर्क कमाल के हैं! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [75]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0


torch.manual_seed(1337)

# !wget https://raw.githubusercontent.com/pytorch/examples/master/word_language_model/data/wikitext-2/train.txt -O input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.237851 M parameters
step 0: train loss 5.7998, val loss 5.7992
step 100: train loss 2.6898, val loss 2.6942
step 200: train loss 2.5243, val loss 2.5230
step 300: train loss 2.4400, val loss 2.4433
step 400: train loss 2.3819, val loss 2.3805
step 500: train loss 2.3230, val loss 2.3304
step 600: train loss 2.2784, val loss 2.2877
step 700: train loss 2.2409, val loss 2.2260
step 800: train loss 2.1985, val loss 2.2064
step 900: train loss 2.1730, val loss 2.1877
step 1000: train loss 2.1388, val loss 2.1543
step 1100: train loss 2.1080, val loss 2.1262
step 1200: train loss 2.0847, val loss 2.0911
step 1300: train loss 2.0744, val loss 2.0678
step 1400: train loss 2.0392, val loss 2.0369
step 1500: train loss 2.0218, val loss 2.0383
step 1600: train loss 2.0025, val loss 2.0174
step 1700: train loss 1.9888, val loss 2.0001
step 1800: train loss 1.9775, val loss 1.9806
step 1900: train loss 1.9596, val loss 1.9521
step 2000: train loss 1.9403, val loss 1.9656
step 2100: train loss 1.

In [ ]:
# Defining a chat function
def chat_with_model(prompt, max_tokens=100):
    """Generate a response to the given prompt using the trained model."""
    # Encode the user input
    encoded_prompt = torch.tensor([encode(prompt)], dtype=torch.long, device=device)

    # Generate a response
    generated_tokens = model.generate(encoded_prompt, max_new_tokens=max_tokens)

    # Decode the generated tokens to text
    response = decode(generated_tokens[0].tolist())
    return response[len(prompt):]  # Return only the newly generated part

# Chat loop
print("ChatGPT-like chatbot ready! Type 'exit' to quit.")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break
    response = chat_with_model(user_input)
    print(f"Chatbot: {response}")


ChatGPT-like chatbot ready! Type 'exit' to quit.
You: What is the main theme of this text?
Chatbot: le in it tra churrecter . 
 
 = = 
 Omanizersful arby Anic tive as a bat with Mark issump belisher '
You: Can you summarize the last paragraph for me?
Chatbot: ter actualtly med 20d Comems : 
 
 Smbels Calleroh Wrimay Crearineists in theirsan , memblishe of ma
You: What is the mood of the current scene
Chatbot: d ip the sing " rink Devdember Simes and Shios the Clicaton <unk> = = = and Zermy A . Crast Isliers 
You: Priers and Aarlone this bannd Cupporty 
Chatbot: at and eles on Atly Releiev comum 's mainnes as triled in " <unk> , reittan to the bramions genumer 
You: commonly referred to as
Chatbot:  Miviry dy . The リhead deamer of the mairs ) out teanlying on the Parmin King , Joroading , Hagh Kup
